In [172]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import erf

%matplotlib inline
#%matplotlib tk

In [15]:
def parameter_set (alpha1, beta1sr, capLam1, rho1, sigma10, alpha2, beta2sr, capLam2, rho2, sigma20, lam, norMu, norStd, meanCo, rhoJ, lamPri, meanId):
    parameters = {
        "alpha1" : alpha1,
        "beta1sr" : beta1sr,
        "capLam1" : capLam1,
        "rho1" : rho1,
        "sigma10" : sigma10,

        "alpha2" : alpha2,
        "beta2sr" : beta2sr,
        "capLam2" : capLam2,
        "rho2" : rho2,
        "sigma20" : sigma20,
        
        
        "lam" : lam,
        "norMu" : norMu,
        "norStd" : norStd,
        
        "meanCo" : meanCo,
        "rhoJ" : rhoJ,
        
        "lamPri" : lamPri,
        "meanId" : meanId}
    
    return parameters

In [23]:
para = parameter_set(1.676, 0.18219, .504, -.964, .16376, 6.488, .21531, 2.115, -1, .07984, .064, -.280, .413, .065, -.52, .013, .052)
para

{'alpha1': 1.676,
 'beta1sr': 0.18219,
 'capLam1': 0.504,
 'rho1': -0.964,
 'sigma10': 0.16376,
 'alpha2': 6.488,
 'beta2sr': 0.21531,
 'capLam2': 2.115,
 'rho2': -1,
 'sigma20': 0.07984,
 'lam': 0.064,
 'norMu': -0.28,
 'norStd': 0.413,
 'meanCo': 0.065,
 'rhoJ': -0.52,
 'lamPri': 0.013,
 'meanId': 0.052}

In [74]:
alpha1 = 1.676
beta1sr = 0.18219 
capLam1 = .504
rho1 = -.964
sigma10 = .16376

alpha2 = 6.488
beta2sr = .21531
capLam2 = 2.115
rho2 =  -1
sigma20 = .07984


lam =  .064
norMu = -.280
norStd = .413

meanCo =  .065
rhoJ =  -.52

lamPri = .013
meanId = .052

alpha = np.array([[np.nan, alpha1, alpha2]]).T
betaSr = np.array([[np.nan, beta1sr, beta2sr]]).T
capLam = np.array([[np.nan, capLam1, capLam2]]).T
rho = np.array([[np.nan, rho1, rho2]]).T
sigma0 = np.array([[np.nan, sigma10, sigma20]]).T


In [224]:
z = np.linspace(0.5, 4, 50) * 1j
z = np.array([z])
z

array([[0.+0.5j       , 0.+0.57142857j, 0.+0.64285714j, 0.+0.71428571j,
        0.+0.78571429j, 0.+0.85714286j, 0.+0.92857143j, 0.+1.j        ,
        0.+1.07142857j, 0.+1.14285714j, 0.+1.21428571j, 0.+1.28571429j,
        0.+1.35714286j, 0.+1.42857143j, 0.+1.5j       , 0.+1.57142857j,
        0.+1.64285714j, 0.+1.71428571j, 0.+1.78571429j, 0.+1.85714286j,
        0.+1.92857143j, 0.+2.j        , 0.+2.07142857j, 0.+2.14285714j,
        0.+2.21428571j, 0.+2.28571429j, 0.+2.35714286j, 0.+2.42857143j,
        0.+2.5j       , 0.+2.57142857j, 0.+2.64285714j, 0.+2.71428571j,
        0.+2.78571429j, 0.+2.85714286j, 0.+2.92857143j, 0.+3.j        ,
        0.+3.07142857j, 0.+3.14285714j, 0.+3.21428571j, 0.+3.28571429j,
        0.+3.35714286j, 0.+3.42857143j, 0.+3.5j       , 0.+3.57142857j,
        0.+3.64285714j, 0.+3.71428571j, 0.+3.78571429j, 0.+3.85714286j,
        0.+3.92857143j, 0.+4.j        ]])

In [77]:
def xi(z):
    
    xi = alpha - rho * capLam @ (1j *z)
    return xi

In [79]:
def dk(z):
    dk = np.sqrt(xi(z) ** 2 + z * (1j + z) * capLam ** 2)
    return dk

In [84]:
def GcoPlus(z):
    GcoPlus = 1 - 1j * z * rhoJ * meanCo - (meanCo / (capLam[1] ** 2)) * (xi(z)[1] + dk(z)[1])
    return GcoPlus

def GcoMinus(z):
    GcoMinus = 1 - 1j * z * rhoJ * meanCo - (meanCo / (capLam[1] ** 2)) * (xi(z)[1] - dk(z)[1])
    return GcoMinus


In [91]:
def GidPlus(z):
    GidPlus = 1 - (meanId / (capLam[1] ** 2)) * (xi(z)[1] + dk(z)[1])
    return GidPlus

def GidMinus(z):
    GidMinus = 1 - (meanId / (capLam[1] ** 2)) * (xi(z)[1] - dk(z)[1])
    return GidMinus


In [95]:
def gk(z):
    gk = (xi(z) - dk(z)) / (xi(z) + dk(z))
    return gk

In [99]:
def AkLogPrice(z, tau = 0.5):
    Ak = (alpha*betaSr**2/(capLam**2))*((xi(z)-dk(z))*tau-2*np.log((1-gk(z)*np.exp(-dk(z)*tau))/(1-gk(z))))
    return Ak

In [104]:
def BkLogPrice(z, tau = 0.5):
    Bk = ((xi(z) - dk(z))*(1 - np.exp(-dk(z)*tau))) / (capLam**2*(1-gk(z)*np.exp(-dk(z)*tau)))
    return Bk

In [109]:
def ThetaCo(z, tau = 0.5):
    theta1 = np.exp(1j*norMu*z - .5*norStd**2*z**2)
    logBlob = np.log((GcoMinus(z)-gk(z)[1]*GcoPlus(z)*np.exp(-dk(z)[1]*tau))/((1-gk(z)[1])*(1-1j*z*rhoJ*meanCo)))
    theta2 = (1/GcoMinus(z))*(1-(2*meanCo*logBlob)/(tau*capLam[1]**2*GcoPlus(z)))
    thetaCo = theta1 * theta2
    return thetaCo

def capCcoPrice(z, tau = 0.5):
    muBar = np.exp(norMu+.5*norStd**2)/(1-meanCo*rhoJ) - 1
    capC = lam*tau*(ThetaCo(z, tau)-1-1j*muBar*z)
    return capC

In [112]:
def ThetaId(z, tau = 0.5):
    logBlob = np.log((GidMinus(z)-gk(z)[1]*GidPlus(z)*np.exp(-dk(z)[1]*tau))/(1-gk(z)[1]))
    thetaId = (1/GidMinus(z))*(1-(2*meanId*logBlob)/(tau*capLam[1]**2*GidPlus(z)))
    return thetaId

def capCidPrice(z, tau = 0.5):
    capC = lamPri*tau*(ThetaId(z, tau)-1)
    return capC

In [126]:
def AkSV(z, tau = 0.5):
    Ak = -2*(alpha*betaSr**2/(capLam**2))*np.log(1-(capLam**2/(2*alpha)*(1-np.exp(-alpha*tau)))@(1j*z))
    return Ak

In [130]:
def BkSV(z, tau=0.5):
    Bk = (np.exp(-alpha*tau)@(1j*z))/(1-(capLam**2/(2*alpha)*(1-np.exp(-alpha*tau)))@(1j*z))
    return Bk

In [135]:
def Theta(z, mu, tau = 0.5):
    logBlob = np.log(1-(1j*z*(capLam[1]**2-2*alpha[1]*mu)*(1-np.exp(-alpha[1]*tau)))/(2*alpha[1]*(1-1j*z*mu)))
    theta = -2*mu/(capLam[1]**2-2*alpha[1]*mu) * logBlob
    return theta

def capCcoSV(z, tau = 0.5):
    capC = lam*Theta(z, meanCo, tau)
    return capC

def capCidSV(z, tau = 0.5):
    capC = lamPri*Theta(z, meanId, tau)
    return capC

In [148]:
tauBar = 30/365
def aK(tauBar = tauBar):
    ak = (1-np.exp(-tauBar*alpha))/alpha
    return ak

def b1(tauBar = tauBar):
    muBar = np.exp(norMu+.5*norStd**2)/(1-meanCo*rhoJ) - 1

    first = (alpha[1]*betaSr[1]**2+lam*meanCo+lamPri*meanId)/alpha[1] * (tauBar-aK(tauBar)[1])
    second = 2*lam*tauBar*(muBar-(norMu+rhoJ*meanCo))
    b1 = first + second
    return b1

def b2(tauBar = tauBar):
    b2 = betaSr[2]**2*(tauBar-aK(tauBar)[2])
    return b2

In [164]:
# Assuming x_t is x_0 = logS_0
S0=40 #Arbitrary choice
x0 = np.log(S0)
r = 0.0
q = 0.0

def CFlogPrice(z, tau = 0.5):
    first = 1j*(x0+(r-q)*tau)*z
    second = AkLogPrice(z, tau)+BkLogPrice(z, tau)*sigma0**2
    secondSum = second[1] + second[2]
    CF = np.exp(first + secondSum + capCcoPrice(z, tau) + capCidPrice(z, tau))
    return CF

def CFSV(z1, z2, tau = 0.5):
    sum1 = AkSV(z1, tau)[1] + BkSV(z1, tau)[1]*sigma0[1]
    sum2 = AkSV(z2, tau)[2] + BkSV(z2, tau)[2]*sigma0[2]
    CF = np.exp(sum1+sum2+capCcoSV(z1, tau)+capCidSV(z1, tau))
    return CF

In [225]:
Z1 = -z * aK()[1] / tauBar
Z2 = -z * aK()[2] / tauBar
I = 10**(-4)
K = 40
firstPart = CFSV(Z1, Z2)
secondPart = np.exp(-1j*z*(b1()+b2()+I)/tauBar) / ((-1j*z)**1.5)
thirdPart = 1 - erf(K/(100*(-1j*z)**0.5))
integ = (firstPart*secondPart*thirdPart).real

In [226]:
a = integ.flatten()

In [227]:
b = z.imag.flatten()

In [233]:
%matplotlib tk
plt.plot(b, a)
plt.grid()

In [206]:
a.shape

(50,)

In [210]:
b.shape

(50,)